<a href="https://colab.research.google.com/github/AniketGaudgaul/AI-ML-DL_Projects/blob/main/Disaster_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download nlp-getting-started
!unzip /content/nlp-getting-started.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/nlp-getting-started.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import numpy as np
import re
import string

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [ ]:
df = pd.read_csv("/content/train.csv")
df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Count the number of NaN values in the 'keyword' column
num_keyword_na = df['keyword'].isna().sum()

# Count the number of NaN values in the 'location' column
num_location_na = df['location'].isna().sum()

print(f"Number of NaN values in 'keyword' column: {num_keyword_na}")
print(f"Number of NaN values in 'location' column: {num_location_na}")

Number of NaN values in 'keyword' column: 61
Number of NaN values in 'location' column: 2533


In [ ]:
# Define function to remove URLs from text
def remove_urls(text):
    # Define regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Remove URLs from text
    text = url_pattern.sub('', text)
    return text

# Apply function to 'text' column
df['text'] = df['text'].apply(remove_urls)

In [ ]:
# Define function to remove unusual characters from text
def remove_unusual_chars(text):
    # Define regular expression pattern to match unusual characters
    unusual_pattern = re.compile(r'[\u0080-\uffff]')

    # Replace unusual characters with a whitespace
    text = unusual_pattern.sub('', text)
    return text

# Apply function to 'text' column
df['text'] = df['text'].apply(remove_unusual_chars)

In [ ]:
# Define function to remove punctuation marks from text
def remove_punctuation(text):
    # Replace punctuation marks with whitespace
    text = re.sub('['+string.punctuation+']', '', text)
    return text

# Apply function to 'text' column
df['text'] = df['text'].apply(remove_punctuation)

In [ ]:
# Convert 'tweets' column to lowercase
df['text'] = df['text'].apply(lambda x: x.lower())

In [ ]:
df['text'][200]

'happening now  hatzolah ems ambulance responding with dual sirens and '

In [ ]:
df['text']

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3       13000 people receive wildfires evacuation orde...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding a bridge collapse int...
7609    ariaahrary thetawniest the out of control wild...
7610                m194 0104 utc5km s of volcano hawaii 
7611    police investigating after an ebike collided w...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

In [ ]:
# Drop 'id' column
df = df.drop('id', axis=1)

In [ ]:
df

,keyword,location,text,target
0,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,all residents asked to shelter in place are be...,1
3,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...
7608,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,NaN,NaN,ariaahrary thetawniest the out of control wild...,1
7610,NaN,NaN,m194 0104 utc5km s of volcano hawaii,1
7611,NaN,NaN,police investigating after an ebike collided w...,1


In [ ]:
# Concatenate the keyword and location columns with the tweet text
df['text'] = df['keyword'].fillna('') + ' ' + df['location'].fillna('') + ' ' + df['text']

# Drop the original keyword and location columns
df.drop(['keyword', 'location'], axis=1, inplace=True)


In [ ]:
# Install the required packages
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import transformers as ppb

# Load the pre-trained BERT model
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Tokenize the input text and convert it to input IDs and attention masks

sentences = df['text'].values
encoded_inputs = tokenizer(sentences.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']
labels = torch.tensor(df['target'].values)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset from input_ids, attention_mask and labels
dataset = TensorDataset(input_ids, attention_mask, labels)

# Create a DataLoader from the TensorDataset
batch_size = 32
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-6)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(1):
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate over batches
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(train_dataloader):
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.last_hidden_state[:, 0, :], labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Track running loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.last_hidden_state[:, 0, :], 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print batch progress
        if batch_idx % 100 == 99:
            print('Epoch: {}, Batch: {}, Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch + 1, batch_idx + 1, running_loss / 100, 100 * correct / total))
            running_loss = 0.0
            correct = 0
            total = 0


Epoch: 1, Batch: 100, Loss: 0.3634, Accuracy: 86.09%
Epoch: 1, Batch: 200, Loss: 0.3262, Accuracy: 86.81%


In [ ]:
df_test = pd.read_csv('test.csv')
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [ ]:
# Define function to remove URLs from text
def remove_urls(text):
    # Define regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Remove URLs from text
    text = url_pattern.sub('', text)
    return text

# Apply function to 'text' column
df_test['text'] = df_test['text'].apply(remove_urls)

# Define function to remove unusual characters from text
def remove_unusual_chars(text):
    # Define regular expression pattern to match unusual characters
    unusual_pattern = re.compile(r'[\u0080-\uffff]')

    # Replace unusual characters with a whitespace
    text = unusual_pattern.sub('', text)
    return text

# Apply function to 'text' column
df_test['text'] = df_test['text'].apply(remove_unusual_chars)

# Define function to remove punctuation marks from text
def remove_punctuation(text):
    # Replace punctuation marks with whitespace
    text = re.sub('['+string.punctuation+']', '', text)
    return text

# Apply function to 'text' column
df_test['text'] = df_test['text'].apply(remove_punctuation)

# Convert 'tweets' column to lowercase
df_test['text'] = df_test['text'].apply(lambda x: x.lower())

# Concatenate the keyword and location columns with the tweet text
df_test['text'] = df_test['keyword'].fillna('') + ' ' + df_test['location'].fillna('') + ' ' + df_test['text']

# Drop the original keyword and location columns
df_test.drop(['keyword', 'location'], axis=1, inplace=True)

In [ ]:

# Preprocess the test data
sentences = df_test['text'].values
encoded_inputs = tokenizer(sentences.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

In [ ]:
df_test

,id,text
0,0,just happened a terrible car crash
1,2,heard about earthquake is different cities s...
2,3,there is a forest fire at spot pond geese ar...
3,9,apocalypse lighting spokane wildfires
4,11,typhoon soudelor kills 28 in china and taiwan
...,...,...
3258,10861,earthquake safety los angeles safety fasten...
3259,10865,storm in ri worse than last hurricane my cit...
3260,10868,green line derailment in chicago
3261,10874,meg issues hazardous weather outlook hwo


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()

predictions = []
with torch.no_grad():
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i+batch_size].to(device)
        batch_attention_mask = attention_mask[i:i+batch_size].to(device)
        with torch.no_grad():
            batch_outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            _, predicted = torch.max(batch_outputs.last_hidden_state[:, 0, :], 1)
            predictions += predicted.cpu().numpy().tolist()


# Save the predictions to a file
df_test['target'] = predictions
df_test[['id', 'target']].to_csv("/content/sample_submission.csv", index=False)


In [ ]:
ans = pd.read_csv("/content/sample_submission.csv")

In [ ]:
ans.head(50)